In [1]:
import pandas as pd

In [2]:
pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.4 MB/s eta 0:00:00


In [8]:
df = pd.read_excel("parsed_data_new_balanced.xlsx")
print(f"📊 Прочитано строк: {len(df)}")

# ---------- Приводим publish_date к дате ----------
df['publish_date'] = pd.to_datetime(df['publish_date'], errors='coerce')

print("Количество пустых дат:", df['publish_date'].isna().sum())

# ---------- Проверка нужных колонок ----------
required_columns = ['project', 'type', 'category', 'title', 'publish_date',
                    'fronturl', 'picture', 'overview', 'text']

if not all(col in df.columns for col in required_columns):
    raise ValueError("❌ В Excel–файле отсутствуют необходимые колонки.")

# ---------- Преобразование DataFrame в список словарей ----------
parsed_messages = df.where(pd.notnull(df), None).to_dict(orient='records')

📊 Прочитано строк: 29334
Количество пустых дат: 1


In [10]:
df = df[df['publish_date'].notna()]

In [11]:
print("Количество пустых дат:", df['publish_date'].isna().sum())

Количество пустых дат: 0


In [12]:
import psycopg2
from psycopg2.extras import execute_values

# ---------- 1. Параметры подключения к БД ----------
db_config = {
    'host': '77.37.164.38',
    'port': 5433,
    'dbname': 'amina',
    'user': 'amina',
    'password': '12345'
}

# ---------- 2. Функция подключения ----------
def init_db():
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    return conn, cursor

# ---------- 3. Функция для быстрой вставки ----------
def insert_to_db(cursor, conn, data):
    if not data:
        print("Нет данных для записи в БД.")
        return

    values = [
        (
            item['project'],
            item['type'],
            item['category'],
            item['title'],
            item['publish_date'],
            item['fronturl'],
            item['picture'],
            item['overview'],
            item['text']
        )
        for item in data
    ]

    insert_query = """
    INSERT INTO stg_news
    (project, type, category, title, publish_date, fronturl, picture, overview, text)
    VALUES %s
    """

    execute_values(cursor, insert_query, values)
    conn.commit()

    print(f"✅ Загружено {len(values)} записей.")

# ---------- 4. Проверка колонок в df ----------
required_columns = ['project', 'type', 'category', 'title', 'publish_date',
                    'fronturl', 'picture', 'overview', 'text']

if not all(col in df.columns for col in required_columns):
    raise ValueError("❌ В DataFrame отсутствуют необходимые колонки.")

# ---------- 5. Подготовка данных: заменяем NaN на None ----------
parsed_messages = df.where(pd.notnull(df), None).to_dict(orient='records')

# ---------- 6. Загрузка в базу ----------
conn, cursor = init_db()
insert_to_db(cursor, conn, parsed_messages)
cursor.close()
conn.close()

print("✅ Все данные успешно загружены в stg_news.")

✅ Загружено 29333 записей.
✅ Все данные успешно загружены в stg_news.


In [14]:
import psycopg2
import pandas as pd

# ---------- 1. Параметры подключения ----------
db_config = {
    'host': '77.37.164.38',
    'port': 5433,
    'dbname': 'amina',
    'user': 'amina',
    'password': '12345'
}

# ---------- 2. Функция подключения ----------
def init_db():
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    return conn, cursor

# ---------- 3. Загружаем данные из базы ----------
conn, cursor = init_db()

# Пишем SQL-запрос (выбери нужные поля)
query = "SELECT project, type, category, title, publish_date FROM stg_news LIMIT 10;"

# Загружаем в DataFrame
df_db = pd.read_sql_query(query, conn)

cursor.close()
conn.close()

# ---------- 4. Смотрим результат ----------
print("📋 Первые записи из базы:")
print(df_db)

<ipython-input-14-7d15a3c62318>:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_db = pd.read_sql_query(query, conn)


📋 Первые записи из базы:
    project        type            category  \
0  Autonews     article                Авто   
1  Autonews  short_news                Авто   
2  Autonews  short_news                Авто   
3  Autonews  short_news                Авто   
4  Autonews  short_news                Авто   
5  Autonews     article                Авто   
6  Autonews     article                Авто   
7       РБК     article  Технологии и медиа   
8       РБК  short_news            Общество   
9  Autonews  short_news                Авто   

                                               title        publish_date  
0  Названы самые популярные иномарки с пробегом. ... 2025-01-10 14:14:15  
1  Подержанные машины разонравились россиянам. Ры... 2025-01-08 13:09:19  
2  Электрокарам продлили бесплатный проезд по «пл... 2025-01-17 14:34:10  
3  Volkswagen обновил кроссовер Talagon. Найдите ... 2025-01-14 09:58:16  
4  В России начали принимать заказы на кроссовер ... 2025-01-24 16:54:00  
5  7 гл